# Quick start with AAanalysis
This is a short introduction to **AAanalysis**, a Python framework for sequence-based protein prediction, centered around the **Comparative Physical Profiling (CPP)** algorithm for interpretable feature engineering.

First, import some third-party packages and ``aanalysis``:

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import aaanalysis as aa
aa.options["verbose"] = False
aa.options["random_state"] = 1

**Data Loading**

We can load a dataset of amino acid scales using the ``aa.load_scales()`` function. Dataset of protein sequences for a binary classification task, such as our γ-secretase substrates (n=63) and non-substrates (n=63) dataset ([Breimann24c]_), can be retrieved via the ``aa.load_dataest()`` function:

In [2]:
# Load example dataset and scales
df_seq = aa.load_dataset(name="DOM_GSEC")
labels = list(df_seq["label"])
df_scales = aa.load_scales()

**Feature Engineering**

To start feature engineering, we can utilize the  ``AAclust`` model for selecting a redundancy-reduced set of amino acid scales:

In [3]:
# Obtain redundancy-reduced set of 100 scales
aac = aa.AAclust()
X = np.array(df_scales).T
scales = aac.fit(X, names=list(df_scales), n_clusters=100).medoid_names_ 
df_scales = df_scales[scales]

We can now use the ``CPP`` algorithm, which aims at identifying a set of features that are most discriminant between two sets of sequences. It´s core idea is the CPP feature concept, defined as a combination of **Parts**, **Splits**, and **Scales**. Parts and Splits can be obtained using ``SequenceFeature``.

We first create a set of baseline features:

In [4]:
sf = aa.SequenceFeature()

# Obtain Parts and Splits
df_parts = sf.get_df_parts(df_seq=df_seq, list_parts=["tmd"])
split_kws = sf.get_split_kws(n_split_max=1, split_types=["Segment"])

Running CPP creates all **Part-Split-Scale** combinations and filters a selected maximum of non-redundant features. As a baseline approach,
we use CPP without filtering (`max_cor=1`) to compute the average values for the 100 selected scales over the entire TMD sequences:

In [ ]:
# CPP creates 100 baseline features (Scale values averaged over TMD)
cpp = aa.CPP(df_scales=df_scales, df_parts=df_parts, split_kws=split_kws)
df_feat_baseline = cpp.run(labels=labels, max_cor=1)
aa.display_df(df=df_feat_baseline, n_rows=10)

By default, ``CPP`` creates around 1000 **Part-Slit** combinations:

In [ ]:
# CPP creates with default parts and splits around 100.000 features and filters them down to 100
df_parts = sf.get_df_parts(df_seq=df_seq)
cpp = aa.CPP(df_scales=df_scales, df_parts=df_parts)
df_feat = cpp.run(labels=labels)
aa.display_df(df=df_feat, n_rows=10)

**Machine Learning**

A feature matrix from a given set of CPP features can be created using ``SequenceFeature.feature_matrix``:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Create feature matrix and perform prediction for baseline features
X = sf.feature_matrix(df_parts=df_parts, features=df_feat_baseline["feature"])
rf = RandomForestClassifier()
cv_base = cross_val_score(rf, X, labels, scoring="accuracy", cv=5)
print(f"Mean accuracy of {round(np.mean(cv_base), 2)}")

Creating more features with CPP will take a little time but improve prediction performance: 

In [ ]:
# Create feature matrix and perform prediction for default CPP features
X = sf.feature_matrix(df_parts=df_parts, features=df_feat["feature"])
rf = RandomForestClassifier()
cv = cross_val_score(rf, X, labels, scoring="accuracy", cv=5) 
print(f"Mean accuracy of {round(np.mean(cv), 2)}")


We can compare the baseline and default CPP feature set using a bar chart:

In [ ]:
# Comparison of baseline and default CPP feature sets
aa.plot_settings()
sns.barplot(pd.DataFrame({"Baseline": cv_base, "CPP": cv}), palette=["tab:blue", "tab:red"])
plt.ylabel("Mean accuracy", size=aa.plot_gcfs()+1)
plt.ylim(0, 1)
plt.title("Comparison of Feature Engineering", size=aa.plot_gcfs()-1)
sns.despine()
plt.show()

**CPP Analysis (group level)**

AAanalysis offers various plotting functions for interpreting the CPP features at group and sample level by ``CPPPlot``. We first need to include the group-level feature importance using the ``TreeModel`` and its ``TreeModel.add_feat_importance`` method:

In [ ]:
tm = aa.TreeModel()
tm.fit(X, labels=labels, use_rfe=False)
df_feat = tm.add_feat_importance(df_feat=df_feat, drop=True)

The top15 features can be displayed using the ``CPPPlot.ranking`` method. The features are shown (left) as a combination of the scale subcategory and their position derived from the part-split combination. The difference of feature values between the test and reference set of protein (here γ-secretase substrates and non-substrates) is shown in the middle. The right subplots displays the feature importance used for ranking:


In [ ]:
# Plot CPP ranking
cpp_plot = aa.CPPPlot()
aa.plot_settings(short_ticks=True, weight_bold=False)
cpp_plot.ranking(df_feat=df_feat)
plt.show()

The distributions of features values for the highest-ranked feature can be displayed using the ``CPPPlot.feature`` method:

In [ ]:
top_feature = df_feat["feature"][0]
top_subcategory = df_feat["subcategory"][0]
aa.plot_settings()
cpp_plot.feature(feature=top_feature , df_seq=df_seq, labels=labels)
plt.title(f"{top_feature} ({top_subcategory})")
plt.show()


The cumulative feature importance per residue position can be visualized by the ``CPPPlot.profile`` method:

In [ ]:
# Plot CPP profile
aa.plot_settings(font_scale=0.9)
cpp_plot.profile(df_feat=df_feat)
plt.show()

 To uncover the common physicochemical signature discriminating the test from the reference protein set, use the ``CPPPlot.feature_map`` method showing the feature value differences per scale subcategory at single-residue resolution:

In [ ]:
# Plot CPP feature map
cpp_plot = aa.CPPPlot()
aa.plot_settings(font_scale=0.65, weight_bold=False)
cpp_plot.feature_map(df_feat=df_feat)
plt.show()

**CPP-SHAP Analysis (sample level)**

We can analyse the impact of features on the prediction score for individual sequence using the ``ShapExplainer`` model, which combines ``CPP`` with the explainable AI framework [SHAP](https://shap.readthedocs.io/en/latest/index.html).

In [ ]:
se = aa.ShapExplainer()
se.fit(X, labels=labels)
df_feat = se.add_feat_impact(df_feat=df_feat, drop=True)
df_feat = se.add_sample_mean_dif(X, labels=labels, df_feat=df_feat, drop=True)

We can now show the feature ranking for a selected protein ('Protein0') using the ``CPPPlot.ranking`` method, plotting SHAP analysis results by setting ``shap_plot=True``:

In [ ]:
aa.plot_settings(short_ticks=True, weight_bold=False)
cpp_plot.ranking(df_feat=df_feat, shap_plot=True, col_dif="mean_dif_Protein0", col_imp="feat_impact_Protein0")
plt.show()

To visualize the CPP-SHAP profile and heatmap, we need to obtain the sequence parts of the respective protein:

In [ ]:
# Get sequences parts for APP
_df_parts = sf.get_df_parts(df_seq=df_seq, list_parts=["tmd", "jmd_c", "jmd_n"])
_args_seq = _df_parts.loc["P05067"].to_dict()
args_seq = {key + "_seq": _args_seq[key] for key in _args_seq}

We can now show the specific **CPP-SHAP Profile** for the first Protein using the ``CPPPlot.profile`` method with setting ``shap_plot=True``:

In [ ]:
# Show CPP-SHAP profile
aa.plot_settings(font_scale=0.9)
cpp_plot.profile(df_feat=df_feat, shap_plot=True, col_imp="feat_impact_Protein0", **args_seq)
plt.show()

With the ``CPPPlot.heatmap`` we can visualize the sample-specific feature value difference and the feature impact per scale subcategory and residue position:   

In [ ]:
# Show CPP heatmap (sample level)
aa.plot_settings(font_scale=0.65, weight_bold=False)
cpp_plot.heatmap(df_feat=df_feat, shap_plot=True, col_val="mean_dif_Protein0", **args_seq)
plt.show()

# Show CPP heatmap (sample level)
cpp_plot.heatmap(df_feat=df_feat, shap_plot=True, col_val="feat_impact_Protein0", **args_seq)
plt.show()

See our [Feature Engineering API](https://aaanalysis.readthedocs.io/en/latest/api.html#feature-engineering) for a comprehensive documentation on the ``CPP``, ``CPPPlot``, ``AAclust``, and ``SequenceFeature`` classes.